#### Optimization & Analytics 23/24

#### Names:

##### - Mohamed Afif Chifaoui (100452024)
##### - Ricardo Vazquez Alvarez (100417117)

---

# Linear and Discrete Models HW1


## Problem statement


A service company operates in three shifts (Morning, Afternoon, Night) and needs to determine the number of employees assigned to each shift. The objective is to minimize labor costs while ensuring sufficient staffing levels for service demand.

Firstly, we can determine that there are two types of sets for the model (the sets of **Shifts** and the **Employee types**). 

1. $I$: Set of Shifts (Morning, Afternoon, Nights)
2. $J$: Set of employee types (Regular, Part-time, Temporary, On-call, Training, Manager, Security).



### Parameters

1. $D_{i}$: Service demand for each shift. ($i \in I$)
2. $E_{j}$: Available number of employees of each type. ($j \in J$)
3. $C_{j}$: Cost for each employee type. ($c \in C$)



The variables are those employees with their type ($j$) to be assigned to a shift ($i$).
- $X_{ij}$: Number of employees of type $j$ assigned to shift $i$.


### Objective function

Minimize the total labor costs per day knowing which type of employee is the best option


$$\text{Minimize} \;\; Z = \sum_{i\in I}^{}\sum_{j\in J}^{} C_{j}\cdot X_{ij}$$



### Decision Variables

The following are the individual decision variables that will be used:

- $x_{11}$: Number of  Regular employees assigned to the Morning shift.
- $x_{12}$: Number of PartTime employees assigned to the Morning shift.
- $x_{13}$: Number of Temporary employees assigned to the Morning shift.
- $x_{14}$: Number of OnCall employees assigned to the Morning shift.
- $x_{15}$: Number of Training employees assigned to the Morning shift.
- $x_{16}$: Number of Manager employees assigned to the Morning shift.
- $x_{17}$: Number of Security employees assigned to the Morning shift.
- $x_{21}$: Number of  Regular employees assigned to the Afternoon shift.
- $x_{22}$: Number of PartTime employees assigned to the Afternoon shift.
- $x_{23}$: Number of Temporary employees assigned to the Afternoon shift.
- $x_{24}$: Number of OnCall employees assigned to the Afternoon shift.
- $x_{25}$: Number of Training employees assigned to the Afternoon shift.
- $x_{26}$: Number of Manager employees for the Afternoon shift.
- $x_{27}$: Number of Security employees for the Afternoon shift.
- $x_{31}$: Number of Regular employees for the Night shift.
- $x_{32}$: Number of PartTime employees on vacation during the Night shift.
- $x_{33}$: Number of Temporary employees on vacation during the Night shift.
- $x_{34}$: Number of OnCall employees on vacation during the Night shift.
- $x_{35}$: Number of Training employees on training during the Night shift.
- $x_{36}$: Number of Manager employees on training during the Night shift.
- $x_{37}$: Number of Security employees on training during the Night shift.


### Constraints


1. Demand constraint: Ensure that offer (total employees in a shift) is higher than demand on a given shift

    $$\sum_{j\in J}^{}  X_{i,j} \geq \sum_{i\in I}^{} D_{i}$$


2. Requisites constraint: Ensure that the number of employees for each employee type chosen does not exceed the amount of employees available.



    $$X_{1, j} \leq E_{j} \;\; \text{for } j \in J$$
    $$X_{2, j} \leq E_{j} \;\; \text{for } j \in J$$
    $$X_{3, j} \leq E_{j} \;\; \text{for } j \in J$$





3. Training regular constraint: Each training employee must have at least 1 regular employee


    $$X_{15}  \leq X_{11} $$
    $$X_{25}  \leq X_{21} $$
    $$X_{35}  \leq X_{31} $$



4. Managers constraint: at least 3 managers per shift

    $$X_{16} \geq 3 $$
    $$X_{26} \geq 3 $$
    $$X_{36} \geq 3 $$


5. Security constraint: at least 1 security employee per shift



    $$X_{17} \geq 1 $$
    $$X_{27} \geq 1 $$
    $$X_{37} \geq 1 $$

4. Non-negativity constraints:

    $$ X_{ij} \geq 0 \quad(\text{for all}\; i\in I,\; j\in J)$$     


### PART A: General formulation

In [1]:
import pandas as pd
import numpy as np
from pyomo.environ import * 
import math

Creating the model, parameters, cost function and the variables.

In [8]:
# The model
model = ConcreteModel()

# Type of different Shifts
model.i = Set(initialize = ['Morning', 'Afternoon', 'Night'], doc = 'Shift')

# The types of different Employees
model.j = Set(initialize = ['Regular', 'PartTime', 'Temp', 'OnCall', 'Training', 'Managers', 'Security'], doc = 'EmployeeType')

# Total amount of workers needed on a shift -> DEMAND
model.total_workers_needed_onshift = Param(model.i, initialize= {'Morning':30, 'Afternoon': 30, 'Night': 30}) 

# Total amount of different workers we have available for each type (each are independent - a worker of one type cannot be of another type) -> AVAILABILITY
model.indiv_workers_have = Param(model.j, initialize={'Regular': 50, 'PartTime': 10, 'Temp': 10, 
                                                      'OnCall': 40, 'Training': 50, 'Managers': 10, 'Security': 5})


# Cost of each worker for each shift -> COST
costperworkershift = {

    # total 21 decision variables we need to solve for 
    # taking into account the cost.
    ('Morning', 'Regular'): 30,
    ('Morning', 'PartTime'): 20,
    ('Morning', 'Temp'): 15,
    ('Morning', 'OnCall'): 25,
    ('Morning', 'Training'): 10,
    ('Morning', 'Managers'): 55,
    ('Morning', 'Security'): 40,

    ('Afternoon', 'Regular'): 30,
    ('Afternoon', 'PartTime'): 20,
    ('Afternoon', 'Temp'): 15,
    ('Afternoon', 'OnCall'): 25,
    ('Afternoon', 'Training'): 10,
    ('Afternoon', 'Managers'): 55,
    ('Afternoon', 'Security'): 40,

    ('Night', 'Regular'): 50,
    ('Night', 'PartTime'): 20,
    ('Night', 'Temp'): 15,
    ('Night', 'OnCall'): 30,
    ('Night', 'Training'): 10,
    ('Night', 'Managers'): 60,
    ('Night', 'Security'): 60,
 

}

# Cost function using the above variable costperworkershift
model.Cost = Param(model.i, model.j, initialize = costperworkershift, doc = 'cost of queries')

Objective function:

    Minimize the cost per day.

In [9]:
# Decision variables we want to solve for
# must be NonNegative since we have that condition (trivial constraint) and must be integers
# (cannot have a decimal amount of a person).
model.x = Var(model.i, model.j, doc = 'Planning', within=NonNegativeIntegers)

# Objective function: minimize the total cost in a day
def objective_rule(model):
    return sum(model.Cost[i,j]*model.x[i,j] for i in model.i for j in model.j)

model.objective = Objective(rule = objective_rule, sense = minimize, doc = 'cost per day')

#### Constraint 1

    Meet the demand for the shift in the day.

In [10]:
# Demand on the shift. Each shift has different demands

def demand(model, i):
    # i = shift, j = employee type 
    # we are adding all the number of employees to be the amount needed on that shift
    return sum(model.x[i,j] for j in model.j)>=model.total_workers_needed_onshift[i]

model.demand = Constraint(model.i, rule = demand, doc= 'employee demand during shift')

#### Constraint 2
    Make sure are not exceeding the number of employees we have of each type.

In [11]:
#Make sure that we are within the constraint of the amount of employees we have

def requisites_rule(model, i, j):
    # Each element in column i for employee type j should be less than or equal to the 
    # available amount.
    return model.x[i, j] <= model.indiv_workers_have[j]

model.requisites_constraint = Constraint(model.i, model.j, rule=requisites_rule)


#### Constraint 3
    Make sure that an employee in training is accompanied by an employee of type regular.

In [12]:
# If we choose a training, we must also have a regular to accompany and train them (1 regular per training)

def training_regular(model, i):
    return model.x[i, 'Training'] <= model.x[i, 'Regular']

model.train_reg = Constraint(model.i, rule = training_regular, doc = 'Each training accompanied by a Regular')

#### Constraint 4
    Must have at least 3 managers at all shifts.

In [13]:
# We need at least 3 managers at all shifts:

def managers_3(model, i):
    return model.x[i, 'Managers'] >= 3

model.managers_3 = Constraint(model.i, rule = managers_3, doc = 'Must have at least 3 managers')

#### Constraint 5
    Need at least one security per shift

In [14]:
# We need at least 1 Security per shift:

def security(model, i):

    # i = shift, j = employee type
    return model.x[i, 'Security'] >= 1

model.security = Constraint(model.i, rule = security, doc = 'Must have at least 1 Security per shift')

Detailed representation of our Pyomo model and its components.

In [15]:
model.pprint()

5 Set Declarations
    Cost_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    i*j :   21 : {('Morning', 'Regular'), ('Morning', 'PartTime'), ('Morning', 'Temp'), ('Morning', 'OnCall'), ('Morning', 'Training'), ('Morning', 'Managers'), ('Morning', 'Security'), ('Afternoon', 'Regular'), ('Afternoon', 'PartTime'), ('Afternoon', 'Temp'), ('Afternoon', 'OnCall'), ('Afternoon', 'Training'), ('Afternoon', 'Managers'), ('Afternoon', 'Security'), ('Night', 'Regular'), ('Night', 'PartTime'), ('Night', 'Temp'), ('Night', 'OnCall'), ('Night', 'Training'), ('Night', 'Managers'), ('Night', 'Security')}
    i : Shift
        Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {'Morning', 'Afternoon', 'Night'}
    j : EmployeeType
        Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    7 : {'Regul

#### Solution to the model

In [16]:
# Define a solver 
Solver = SolverFactory('glpk')

# Obtain the solution
Results = Solver.solve(model)

# Display the solution
model.x.display()

x : Planning
    Size=21, Index=x_index
    Key                       : Lower : Value : Upper : Fixed : Stale : Domain
    ('Afternoon', 'Managers') :     0 :   3.0 :  None : False : False : NonNegativeIntegers
      ('Afternoon', 'OnCall') :     0 :   0.0 :  None : False : False : NonNegativeIntegers
    ('Afternoon', 'PartTime') :     0 :   0.0 :  None : False : False : NonNegativeIntegers
     ('Afternoon', 'Regular') :     0 :   8.0 :  None : False : False : NonNegativeIntegers
    ('Afternoon', 'Security') :     0 :   1.0 :  None : False : False : NonNegativeIntegers
        ('Afternoon', 'Temp') :     0 :  10.0 :  None : False : False : NonNegativeIntegers
    ('Afternoon', 'Training') :     0 :   8.0 :  None : False : False : NonNegativeIntegers
      ('Morning', 'Managers') :     0 :   3.0 :  None : False : False : NonNegativeIntegers
        ('Morning', 'OnCall') :     0 :   0.0 :  None : False : False : NonNegativeIntegers
      ('Morning', 'PartTime') :     0 :   0.0 :  None

With the final cost being the objective function value:

In [17]:
# Total amount of money spent
model.objective()

2120.0

### INTERPRETATION AND ANALYSIS

For the 21 variables we have we can see that the **solution** is the following:

- 3 Managers for the Morning shift ($x_{16}$)
- 8 Regular employees for the Morning shift ($x_{11}$)
- 1 Security employee for the Morning shift ($x_{17}$)
- 10 Temporary employees for the Morning shift ($x_{13}$)
- 8 Training employees for the Morning shift ($x_{15}$)


- 3 Managers for the Afternoon shift ($x_{26}$)
- 8 Regular employees for the Afternoon shift ($x_{21}$)
- 1 Security employee for the Afternoon shift ($x_{27}$)
- 10 Temporary employees for the Afternoon shift ($x_{23}$)
- 8 Training employees for the Afternoon shift ($x_{25}$)



- 3 Managers for the Night shift ($x_{36}$)
- 10 PartTime employees for the Night shift ($x_{32}$)
- 3 Regular employees for the Night shift ($x_{31}$)
- 1 Security employee for the Night ($x_{37}$)
- 10 Temporary employees for the Night shift ($x_{33}$)
- 3 Training employees for the Night shift ($x_{35}$)


In the end, a total of **2120 euros** will be spent to cover this service for one day.


### Analysis of Multiple days:

Now we can do this more systematically by placing all the previous code on the creation of the model into a function. This way we can simulate an entire **week's schedule** depending on the **demand per shift each day** and measure the different **costs** of each day. 

To do so it is also convenient to create a dataframe for the solution to the model including in the rows, the **shift type** and the column the **employee type**, where the actual data of the model is the number of employees chosen for the shift type and employee type (decision variables solution).

It is done in the following way:

1. **ComputeModel(demandonday:list)** $\rightarrow$ takes in a list for the demand on each shift type (Morning, Afternoon, Night).

2. **obtain_df_solution(model)** $\rightarrow$ takes in the model and converts the results into a dataframe.


In [36]:
def obtain_df_solution(model):
    

    """
    Creates a Pandas DataFrame representing the solution to the Pyomo model.

    Parameters:
    - model (pyomo.environ.ConcreteModel): Completed Pyomo model with constraints, parameters,
      variables, cost, and objective function.

    Returns:
    - pd.DataFrame: Pandas DataFrame where rows represent shifts and columns represent
      employee types. The DataFrame includes the solution to the decision variables.
      
    """
    names = ['Morning', 'Afternoon', 'Night']
    shifts = ['Regular', 'PartTime', 'Temp', 'OnCall', 'Training', 'Managers', 'Security']

    index_order = list(model.x.index_set())
    values_in_order = [model.x[i]() for i in index_order]

    values_array = np.empty((0, 7))


    for i in range(0,len(names)):
        values_array = np.append(values_array, [values_in_order[len(shifts)*i:len(shifts)*(i+1)]], axis=0)


    # creating the df
    df = pd.DataFrame(values_array, index=names, columns=shifts)

    Solver = SolverFactory('glpk')

    return df

Insert the model into a Pandas DataFrame

In [19]:
df = obtain_df_solution(model)
df

,Regular,PartTime,Temp,OnCall,Training,Managers,Security
Morning,8.0,0.0,10.0,0.0,8.0,3.0,1.0
Afternoon,8.0,0.0,10.0,0.0,8.0,3.0,1.0
Night,3.0,10.0,10.0,0.0,3.0,3.0,1.0


In [37]:
def ComputeModel(demandonday:list):

    """
    Generates a completed Pyomo model that accommodates the given demands for each shift per day.

    Parameters:
    - demandonday (list of int): A list of integers representing the demands on all shifts per day.
      (e.g., demandonday = [Morning demand, Afternoon demand, Midnight demand])

    Returns:
    - pyomo.environ.ConcreteModel: A completed Pyomo model that accommodates the specified demands.
    """

    # Create a list of names for the shifts
    names = ['Morning', 'Afternoon', 'Night']

    # create the dictionary needed with a comprehension format
    initializeworkersneeded = {str(key):value for key,value in zip(names,demandonday)}


    model = ConcreteModel()


    model.i = Set(initialize = names, doc = 'Shift')

    # The types of different Employee
    model.j = Set(initialize = ['Regular', 'PartTime', 'Temp', 'OnCall', 'Training', 'Managers', 'Security'], doc = 'EmployeeType')

    # Total amount of workers needed on a shift -> DEMAND
    model.total_workers_needed_onshift = Param(model.i, initialize= initializeworkersneeded) 

    # Total amount of different workers we have for each type (each are independent - a worker of one type cannot be of another type) -> AVAILABILITY
    model.indiv_workers_have = Param(model.j, initialize={'Regular': 50, 'PartTime': 10, 'Temp': 10, 
                                                        'OnCall': 40, 'Training': 50, 'Managers': 10, 'Security': 5})


    # Cost of each worker for each shift.
    costperworkershift = {

        # total 21 decision variables
        ('Morning', 'Regular'): 30,
        ('Morning', 'PartTime'): 20,
        ('Morning', 'Temp'): 15,
        ('Morning', 'OnCall'): 25,
        ('Morning', 'Training'): 10,
        ('Morning', 'Managers'): 55,
        ('Morning', 'Security'): 40,


        ('Afternoon', 'Regular'): 30,
        ('Afternoon', 'PartTime'): 20,
        ('Afternoon', 'Temp'): 15,
        ('Afternoon', 'OnCall'): 25,
        ('Afternoon', 'Training'): 10,
        ('Afternoon', 'Managers'): 55,
        ('Afternoon', 'Security'): 40,


        ('Night', 'Regular'): 50,
        ('Night', 'PartTime'): 20,
        ('Night', 'Temp'): 15,
        ('Night', 'OnCall'): 30,
        ('Night', 'Training'): 10,
        ('Night', 'Managers'): 60,
        ('Night', 'Security'): 60,

    }

    # Cost function using the above costperworkershift
    model.Cost = Param(model.i, model.j, initialize = costperworkershift, doc = 'cost of queries')


    # Decision variables we want to solve for
    # must be NonNegative since we have that condition (trivial constraint) and must be integers
    # (cannot have a decimal amount of a person)

    model.x = Var(model.i, model.j, doc = 'Planning', within=NonNegativeIntegers)


    # we want to minimize the total cost in a day
    def objective_rule(model):
        return sum(model.Cost[i,j]*model.x[i,j] for i in model.i for j in model.j)

    model.objective = Objective(rule = objective_rule, sense = minimize, doc = 'cost per day')


    def demand(model, i):

        # i = shift, j = employee type 
        # we are adding all the number of employees to be the amount needed on that shift
        return sum(model.x[i,j] for j in model.j)>=model.total_workers_needed_onshift[i]

    model.demand = Constraint(model.i, rule = demand, doc= 'employee demand during shift')


    def requisites_rule(model, i, j):
        
        # Each element in column i for employee type j should be less than or equal to the 
        # available amount.
        return model.x[i, j] <= model.indiv_workers_have[j]

    model.requisites_constraint = Constraint(model.i, model.j, rule=requisites_rule, doc = 'Available amount of employees for type')

    # If we choose a training, we must also have a regular to accompany and train them (1 regular per training)

    def training_regular(model, i):

        return model.x[i, 'Training'] <= model.x[i, 'Regular']

    model.train_reg = Constraint(model.i, rule = training_regular, doc = 'Each training accompanied by a Regular')

    # We need at least 3 managers at all shifts:
    def managers_3(model, i):
        return model.x[i, 'Managers'] >= 3

    model.managers_3 = Constraint(model.i, rule = managers_3, doc = 'Must have at least 3 managers')

    # We need at least 1 Security per shift
    def security(model, i):

        # i = shift, j = employee type
        return model.x[i, 'Security'] >= 1

    model.security = Constraint(model.i, rule = security, doc = 'Must have at least 1 Security per shift')

    Results = Solver.solve(model)
    
    obj = model.objective()

    
    return model,obj

The following code runs a simulation for an entire week in order to compute the model and objective value of each of the days (7 days). That is to say that each day of the week has its own demand (**'demandonday'** list) which computes its own model and cost of the day.

Therefore, the following inserts the **demand** values for each shift in a single day of the week. Then it computes the **model** and stores the results in a dataframe using the **obtain_df_solution** function previously created.

In [21]:
# All the data on Monday
demandMonday  = [30,30,30]
modelMonday,objectiveMonday = ComputeModel(demandMonday)
dfMonday = obtain_df_solution(modelMonday)

# All the data on Tuesday
demandTuesday= [30,35,30]
modelTuesday, objectiveTuesday  = ComputeModel(demandTuesday)
dfTuesday= obtain_df_solution(modelTuesday)

# All the data on Wednesday
demandWednesday= [35,40,45]
modelWedn, objectiveWedn  = ComputeModel(demandWednesday)
dfWedn= obtain_df_solution(modelWedn)

# All the data on Thursday
demandThur= [35,40,40]
modelThur, objectiveThur  = ComputeModel(demandThur)
dfThur= obtain_df_solution(modelThur)

# All the data on Friday
demandFri= [40,50,55]
modelFri, objectiveFri  = ComputeModel(demandFri)
dfFri= obtain_df_solution(modelFri)

# All the data on Saturday
demandSat= [50,60,60]
modelSat, objectiveSat  = ComputeModel(demandSat)
dfSat= obtain_df_solution(modelSat)

# All the data on Sunday
demandSun= [50,60,60]
modelSun, objectiveSun  = ComputeModel(demandSun)
dfSun= obtain_df_solution(modelSun)



### Visualization

In [22]:
%%HTML
<script src="require.js"></script>

In [23]:
import plotly.express as px
import matplotlib.pyplot as plt


# Create a list of dataframes and objectives(cost function)
dfs = [dfMonday, dfTuesday, dfWedn, dfThur, dfFri, dfSat, dfSun]
objectives = [objectiveMonday, objectiveTuesday, objectiveWedn, objectiveThur, objectiveFri, objectiveSat, objectiveSun]
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

# Create a DataFrame for cost data
cost_df = pd.DataFrame({'Day': days, 'Cost': objectives})

# Use plotyl for an interactive plot
fig = px.bar(cost_df, x='Day', y='Cost', text='Cost', title='Cost for Each Day of the Week')
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(yaxis_title='Objective Value (Cost)', xaxis_title='Day of the Week')

fig.show()



As we can see in the above plot, **cost** of the service increases on Friday and on the weekend which is due to a **higher demand** on these days when compared to weekdays. In general, there is an obvious positive linear relationship between both the Cost and the demand for each day. This means that as the demand increases (we need more employees) then the cost will also increase.

## Part C: Sensitivities associated with each constraint, and interpretion of the obtained values.

**Because the model.dual() function does not work for our model (it produced an error), then the sensitivity needs to be calculated separately.**

The sensitivity is done in general to find how much the objective function changes when changing the parameter it depends on by one unit. Of course, there are three parameters in the model which might also be used for calculating the sensitivity. But in this case, the only parameter that can change is the demand for a certain day and shift. This is because the other ones have been defined to be constant.

The model.dual() function would run a simulation on the already created model changing its parameter by one unit and examining the change in the objective function for multiple iterations. Then obtain the average. But in this case, the model.dual() function cannot do so since there are three parameters, where each parameter contains a list of values, and therefore cannot do a sensitivity analysis.

This is done the following way:

**1. Define a new column in the data frame of a day the cost per shift that was obtained.**

**2. Calculate the sensitivity for each shift on day of the week using the following formula:**

$$\text{Sensitivity}_{\text{day 2 - day 1}} = \dfrac{1}{N} \;\sum^{N=3}_{\text{shift} = 1} \left( \dfrac{\Delta \;(\text{Cost})_{\text{shift}}}{\Delta (\text{Demand})_{\text{shift}}} \right)$$

    Where we take the average of the sensitivities per shift to get the average sensitivity in a day.

The sensitivity per shift would be the difference between demands of a shift for two days and the same for the cost. 

**For example:**

- Thursday demand = [Morning =35, Afternoon = 40, Night = 40]
- Friday demand = [Morning = 40, Afternoon = 50, Night = 55]

**Sensitivity between Tuesday and Monday (done for just the Morning shift):**


<br>
<br>
Calculating the cost comes from the solution to the model (decision variables):

$$\Delta \text{Cost}_{\text{Morning}} = \text{Cost}_{\text{Tuesday, Morning}} - \text{Cost}_{\text{Monday, Morning}} = 100$$


<br>
<br>
The demand is predefined:

$$\Delta \text{Demand}_{\text{Morning}} =   \text{Demand}_{\text{(Tuesday, Morning)}} - \text{Demand}_{\text{(Monday, Morning)}} = 5$$

<br>

Calculating sensitivity:

$$\text{Sensitivity}_{\text{Morning}} = \dfrac{100}{5} = 20$$
<br>

    Then repeat for the rest of the shifts and obtain the average between all.

**3. Take the average of all sensitivities for all days.**

- The above sensitivity calculations will be done for the different data used for the demand on each shift, which was already created before when doing an analysis on the cost difference per day of the week. Therefore, seeing on average how much the cost changes when we change the demand parameter will give the general sensitivity. 

We are only calculating the sensitivity with respect to the only parameter that changes (the demand per shift on a certain day).

Firstly we need to define a function that can systematically add a total cost of the shift to the dataframe that was created before for each day. We also need to define the data used (cost of each employee type for each shift).

In [33]:

# dataframe friendly dictionary for cost per shift
data = {'Regular': [30, 30, 50],
        'PartTime': [20, 20, 20],
        'Temp': [15, 15, 15],
        'OnCall': [25, 25, 30],
        'Training': [10, 10, 10],
        'Managers': [55, 55, 60],
        'Security': [40, 40, 60]
        }

def addCostPerShift(modeldf: pd.DataFrame, data:dict):

    """
    Adds a new column to the Pandas DataFrame 'modeldf' with the total cost per each shift.

    Parameters:
    - modeldf (pd.DataFrame): Pandas DataFrame containing the solution to the model (the decision variables).
    - data (dict): Dictionary containing the data of the cost per shift.

    This function modifies 'modeldf' in place and does not return anything.
    """
    # if the column has already been added. Do not add it again
    if 'Total Shift Cost' in modeldf:
        return
    # Add the column if it does not exists
    else:
        
        # new data frame with the values for the cost
        df = pd.DataFrame(data)
        df.set_index(pd.Index(['Morning', 'Afternoon', 'Night']), inplace=True)
        listofsums = []

        # Calculating the total cost.
        for i in range(0,len(modeldf)):
            mysum=0
            for j in range(0,len(modeldf.columns)):
                mysum += modeldf.iloc[i,j]*df.iloc[i,j]
            listofsums.append(mysum)

        modeldf['Total Shift Cost'] = listofsums

        # returns nothing just adds a new column to the data frame
        return

In [34]:
# an example for the tuesday data
addCostPerShift(dfTuesday,data)
dfTuesday

,Regular,PartTime,Temp,OnCall,Training,Managers,Security,Total Shift Cost
Morning,8.0,0.0,10.0,0.0,8.0,3.0,1.0,675.0
Afternoon,10.0,1.0,10.0,0.0,10.0,3.0,1.0,775.0
Night,3.0,10.0,10.0,0.0,3.0,3.0,1.0,770.0


##### Part 1

Adding the new column of total cost per shift type to all days defined (Monday $\rightarrow$ Sunday).


In [35]:
all_demands = [demandMonday, demandTuesday, demandWednesday, demandThur, demandFri, demandSat, demandSun]

all_df = [dfMonday, dfTuesday, dfWedn, dfThur, dfFri, dfSat, dfSun]

# adding the new column, total cost per shift type to the dataframe.
for i in range(0,len(all_df)):
    addCostPerShift(all_df[i],data)

##### Part 2

Calculating Sensitivity using the equation created.

In [39]:
def sensitivities(demand_i:list, df_i:pd.DataFrame, demand_f:list, df_f:pd.DataFrame):

    """
    Calculates the relative sensitivity between two scenarios.

    Parameters:
    - demand_i (list of int): Initial demand on a day, including specific demands for each shift.
    - df_i (pd.DataFrame): Pandas DataFrame with the cost per shift column for the initial scenario.
    - demand_f (list of int): Final demand on a day, including specific demands for each shift.
    - df_f (pd.DataFrame): Pandas DataFrame with the cost per shift column for the final scenario.

    Returns:
    - float: The relative sensitivity between the two scenarios.
    """

    # demand_i is the initial day to compare it to
    # costshift_i is the initial day cost per shift column in a list
    # meanwhile subscript f is for final

    costshift_f = df_f['Total Shift Cost'].tolist()
    costshift_i = df_i['Total Shift Cost'].tolist()

    
    delta_demand = np.array(demand_f)-np.array(demand_i)

    delta_cost = np.array(costshift_f) - np.array(costshift_i)

    # to make sure that if the numerator or denominator is 0 then just return 0
    # division of 0 would give a Nan
    sensitivity_day = np.mean(delta_cost/delta_demand)

    if math.isnan(sensitivity_day):
        return 0
    else:
        return sensitivity_day

In the following code, we encounter a RuntimeWarning. This is because it attempts to divide by zero. 

In the above function, the division of zero occurs and as a result the average sensitivity for the day become NaN. Therefore, if the value is NaN then an if statement just returns 0 rather than the NaN.

In [41]:
all_sens = np.zeros(len(all_df))

for i in range(1, len(all_df)):
    with np.errstate(divide='ignore', invalid='ignore'):
        all_sens[i-1] = sensitivities(all_demands[i-1], all_df[i-1], all_demands[i], all_df[i])

##### Part 3

    Taking the average of all sensitivities for each day of the week to get a final sensitivity.

Because we have days of the week where on average there are no differences (Monday and Tuesday have the same demand) then we do not want to include it in the sensitivity calculation.

In [42]:
# Showing all the individual sensitivities for each day
all_sens

array([ 0.        , 23.33333333,  0.        , 23.33333333, 23.33333333,
        0.        ,  0.        ])

In [43]:
# Making sure that if the difference between 2 days was 0
# then we do not need to use it for the calculation of the sensitivity.
N = 0
for i in range(0,len(all_sens)):
    if all_sens[i] != 0:
        N+=1
    else:
        continue
    
# if statement that shows if N is 0 then it does not divide by N and only shows the
# sum of all values.
if N != 0 : sensitivity=sum(all_sens)/N 
else: sensitivity = sum(all_sens) 

In [31]:
print("The average sensitivity found was: {:.2f}".format(sensitivity))

The average sensitivity found was: 23.33


The above value of sensitivity resulted in 23.33. This means that as we increase a unit of average demand for the day (increase on average each shift type by 1 unit or employee needed) then the cost also will increase by 23.33. 

### Part D: Binary Variables

Modify the problem in (a) to impose some logical and conditional constraints that require the use of binary or integer variables. The more integer variables or constraints, the better. Implement and solve this new model and interpret the results.

#### STATEMENT

In this new case, we have the profit per employee type and how much does the employee cost to the company in a given day of the week. We aim to maximize the profit and know which employee type is the best option according to our economic capacity (constraints) in a specific day of the week.

| $_\text{Employee Type}$  $^{\text{Day of week}}$ | Monday | Tuesday | Wednesday | Thursday | Friday | Saturday | Sunday | Profit |
|---------------|---------|-----------|-------|-----|-----|-----|-----|-----|
| 1- Regular       | 15 | 15 | 15 |15 | 20 | 20 | 20 | 200 |  
| 2- Part-time     | 10 | 10 | 10 | 10 | 15 | 15 | 15 | 130|  
| 3- Temporary     | 8 | 8 | 8 | 8 | 12 | 12 | 12 | 115 | 
| 4- On-call       | 9 | 9 | 9 | 9 | 13 | 13 | 13 | 95 |  
| 5- Training      | 4 | 4 | 4 |4 |8 | 8 | 8 | 80 |  
| 6- Manager       | 18 | 18 | 18 | 18 | 18 | 23 | 23 | 225|    


Day | Budget
 -------- | ------ 
Monday | 50
Tuesday | 50
Wednesday | 50
Thursday | 50
Friday | 80
Saturday | 80
Sunday | 80


The problem we are addressing can be formulated using the following model:  
 
<li> <font color="blue">Decision variables:</font>

$$x_i =
\left\{\begin{array}{ll} 
1, & \text{if employee type $i$ is selected,}\\
0, & \text{if employee type $i$ is not selected}
\end{array} \right.\quad i=1,2,3,4,5,6$$

<li> <font color="blue">Objective:</font> maximize profit.

$$
200 x_1 + 130 x_2 + 115 x_3 + 95 x_4 + 80 x_5 + 225 x_6
$$



<li><font color="blue">Constraints:</font>
<ul>

<li>Budget per day:

$$
\begin{array}{ll}
15 x_1 + 10 x_2 + 8 x_3 + 9 x_4 + 4 x_5 + 18 x_6 \leq 50 
\\
15 x_1 + 10 x_2 + 8 x_3 + 9 x_4 + 4 x_5 + 18 x_6 \leq 50
\\
15 x_1 + 10 x_2 + 8 x_3 + 9 x_4 + 4 x_5 + 18 x_6 \leq 50 
\\
15 x_1 + 10 x_2 + 8 x_3 + 9 x_4 + 4 x_5 + 18 x_6 \leq 50 
\\
20 x_1 + 15 x_2 + 12 x_3 + 13 x_4 + 8 x_5 + 23 x_6 \leq 80 
\\
20 x_1 + 15 x_2 + 12 x_3 + 13 x_4 + 8 x_5 + 23 x_6 \leq 80 
\\
20 x_1 + 15 x_2 + 12 x_3 + 13 x_4 + 8 x_5 + 23 x_6 \leq 80 
\end{array}
$$



<li>Binary variables:

$$
x_i \in \{ 0,1\}, i=1,2,3,4,5,6
$$
</ul>

 
  

In [26]:

from pyomo.environ import *
# import gurobi
model = ConcreteModel()

# Variables
model.x = Var([1,2,3,4,5,6], domain=Binary) 

# Objective function
model.OBJ = Objective(expr=200*model.x[1] + 130*model.x[2] + 115*model.x[3] + 95*model.x[4] + 80*model.x[5] + 225*model.x[6], sense=maximize)


# Constraints for each day of the week 1-7(Mon-Sun)

# Monday
model.cons1 = Constraint(expr=15*model.x[1] + 10*model.x[2] + 8*model.x[3] + 9*model.x[4] + 4*model.x[5] + 20*model.x[6] <=50)
# Tuesday
model.cons2= Constraint(expr=15*model.x[1] + 10*model.x[2] + 8*model.x[3] + 9*model.x[4] + 4*model.x[5]+ 20*model.x[6] <=50)
# Wednesday
model.cons3 = Constraint(expr=15*model.x[1] + 10*model.x[2] + 8*model.x[3] + 9*model.x[4] + 4*model.x[5]+ 20*model.x[6] <=50)
# Thursday
model.cons4 = Constraint(expr=15*model.x[1] + 10*model.x[2] + 8*model.x[3] + 9*model.x[4] + 4*model.x[5]+ 20*model.x[6] <=50)
# Friday
model.cons5 = Constraint(expr=20*model.x[1] + 15*model.x[2] + 12*model.x[3] + 13*model.x[4] + 8*model.x[5]+ 25*model.x[6] <=80)
# Saturday
model.cons6 = Constraint(expr=20*model.x[1] + 15*model.x[2] + 12*model.x[3] + 13*model.x[4] + 8*model.x[5]+ 25*model.x[6] <=80)
# Sunday
model.cons7 = Constraint(expr=20*model.x[1] + 15*model.x[2] + 12*model.x[3] + 13*model.x[4] + 8*model.x[5]+ 25*model.x[6] <=80)




Solver = SolverFactory('glpk')

# Solver = SolverFactory('gurobi')

Results = Solver.solve(model)

# Display solution
display(model.OBJ)


OBJ : Size=1, Index=None, Active=True
    Key  : Active : Value
    None :   True : 635.0


In [27]:
display(model)

Model unknown

  Variables:
    x : Size=6, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :   1.0 :     1 : False : False : Binary
          2 :     0 :   1.0 :     1 : False : False : Binary
          3 :     0 :   0.0 :     1 : False : False : Binary
          4 :     0 :   0.0 :     1 : False : False : Binary
          5 :     0 :   1.0 :     1 : False : False : Binary
          6 :     0 :   1.0 :     1 : False : False : Binary

  Objectives:
    OBJ : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 635.0

  Constraints:
    cons1 : Size=1
        Key  : Lower : Body : Upper
        None :  None : 49.0 :  50.0
    cons2 : Size=1
        Key  : Lower : Body : Upper
        None :  None : 49.0 :  50.0
    cons3 : Size=1
        Key  : Lower : Body : Upper
        None :  None : 49.0 :  50.0
    cons4 : Size=1
        Key  : Lower : Body : Upper
        None :  None : 49.0 :  50.0
    cons5 : Size=1

From the **solution** we can see that the service will hire the following **employees**:

- 1 Regular employee ($x_1$)
- 1 Part-time employee ($x_2$)
- 1 Training employee ($x_5$)
- 1 Manager employee ($x_6$)

And the total **profit** will be equal to **635 euros**

## Conclusions

In this first project we gained much knowledge and insights about linear and discrete models, apart from the mathematical background behind these concepts learnt in the theoretical classes we were able to develop a simple solution for a problem that many companies can face in their day-to-day which is workforce scheduling and hiring in order to maximize their financial return.
We still think that there is still room for improvement in case it is a more complex problem, but we are happy with the obtained results. Looking forward to the next challenge!